In [1]:
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 8.6 MB/s eta 0:00:00


In [57]:
# Begin with the general recursive Solovay-Kitaev framework as given in Dawson & Nielsen 2005

import numpy as np
import math



def inverse_sequence(seq):
  out = ''.join(seq.swapcase() for char in seq)
  return out[::-1]

def dagger(A: np.ndarray):
    return A.conj().T

# Return the sequence of gates to construct the unitary + the actual approximation matrix itself
def solovay_kitaev(U,n): # input :: gate , depth
  if(n==0): # base case
    return enet_approx(U) # returns the sequence
  U_n_1_seq, U_n_1 = solovay_kitaev(U, n-1)
  #U_prev = multiply(U_prev_seq)
  print(U @ dagger(U_n_1))
  V,W = GC_decompose(U @ dagger(U_n_1))
  V_n_1_seq, V_n_1 = solovay_kitaev(V,n-1)
  W_n_1_seq, W_n_1 = solovay_kitaev(W,n- 1)
  V_inv_seq = inverse_sequence(V_n_1_seq)

  W_inv_seq = inverse_sequence(W_n_1_seq)


  U_n = V_n_1 @ W_n_1 @ dagger(V_n_1) @ dagger(U_n_1) @ U_n_1
  U_n_seq = V_n_1_seq + W_n_1_seq + inverse_sequence(V_n_1_seq) + inverse_sequence(W_n_1_seq) + U_n_1_seq
  return U_n_seq, U_n

In [75]:
# Building an initial enet
enet = []
enet_strs = []

def build_enet(gateset, max_depth, gateset_desc):

  global enet, enet_strs
  print(max_depth, len(enet))
  if max_depth <= 0:
    return
  if len(enet) == 0:
    enet = list(gateset.copy())
    enet_strs = list(gateset_desc.copy())
    build_enet(gateset, max_depth - 1, gateset_desc)
  else:
    # Take every element of the enet and left multiply it by every element in the gateset
    for i in range(len(gateset)):
      gate = gateset[i]

      current = len(enet)
      for j in range(current):

        thing = gate @ enet[j]
        if np.any(np.all(enet == thing, axis = (1, 2))):
          continue

        enet.append(gate @ enet[j])
        enet_strs.append(gateset_desc[i] + enet_strs[j])

    build_enet(gateset, max_depth - 1, gateset_desc)

def enet_approx(U):
  trace_norms = np.linalg.norm(enet - U, ord='nuc', axis=(1, 2))

  # Find the index of the matrix in the array closest in trace norm to the given matrix
  closest_index = np.argmin(np.abs(trace_norms))

  return enet_strs[closest_index], enet[closest_index]


# This is a rough approximation for an actual enet that just brute forces through all gateset words up to depth max_depth
build_enet(np.array((np.array([[1, 2], [0, 1]]), np.array([[1, 2], [2, 2]]))), 5, ['a', 'b'])
enet = np.array(enet)
#print(enet, enet_strs)
print(enet_approx(np.array([[1, 0], [0, 1]])))



5 0
4 2
3 8
2 24
1 66
0 176
('a', array([[1, 2],
       [0, 1]]))


In [79]:
# Turn arbitrary unitary matrix into special unitary by normalizing determinant
def SU2(U):
  determinant = np.linalg.det(U)
  return (1/determinant)**0.5 * U

def Rx(phi):
  return np.array([[np.cos(phi/2), -1j*np.sin(phi/2)], [-1j*np.sin(phi/2), np.cos(phi/2)]])

def Ry(phi):
  return np.array([[np.cos(phi/2), -np.sin(phi/2)], [np.sin(phi/2), np.cos(phi/2)]])


def GC_decompose(U):
  print(U[0][0].real)
  theta = 2*math.acos(U[0][0].real)

  phi = 2*math.asin(math.sqrt(math.sqrt(0.5*(1+math.sqrt(1 - math.sin(theta/2.0)**2)))))
  V = SU2(Rx(phi))
  W = SU2(Ry(phi))
  Winv = dagger(W)
  Vinv = dagger(V)
  comm = V @ W @ Vinv @ Winv
  w1,v1= np.linalg.eig(U)
  w2,v2= np.linalg.eig(comm)

  S = SU2(v1 @ dagger(v2))

  Vout = S @ V @ dagger(S)
  Wout = S @ W @ dagger(S)
  return Vout, Wout

In [64]:
# Test
V, W = GC_decompose([[0, 1j], [1, 0]])
print(V @ W @ dagger(V) @ dagger(W))  # phase invariant?, idk

[[-6.66133815e-16+2.77555756e-16j  7.07106781e-01-7.07106781e-01j]
 [-7.07106781e-01-7.07106781e-01j  0.00000000e+00-5.55111512e-17j]]


In [77]:
# Test Solovay-Kitaev -- clifford + T synthesis

# First, build approximate enet from gateset
pauli_gateset = [[[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]]
clifford_t_gateset = [
  [[np.sqrt(1/2), -np.sqrt(1/2)], [np.sqrt(1/2), -np.sqrt(1/2)]],
  [[1, 0], [0, 1j]],
  [[1, 0], [0, np.cos(np.pi/4)+1j*np.sin(np.pi/4)]]
                      ]
clifford_t_labels = ['H', 'S', 'T']
clifford_t_gateset.extend([dagger(np.array(i)) for i in clifford_t_gateset])
clifford_t_labels += ['h', 's', 't']
enet = []
enet_strs = []
build_enet(np.array(clifford_t_gateset), 3, clifford_t_labels)



3 0
2 6
1 260
0 8351


In [81]:
enet_dict = {'enet': enet, 'enet_strs': enet_strs}

In [82]:
import _pickle as cPickle

with open(r"enet.pickle", "wb") as output_file:
  cPickle.dump(enet_dict, output_file)


In [59]:
# [[g, h], [h, g^-1]]
def elkasapy_commutator(U, V):
  Vinv = dagger(V)
  Uinv = dagger(U)
  return U @ V @ Uinv @ Uinv @ Vinv @ U @ V @ U @ Vinv @ Uinv @ Uinv @ V @ U @ Vinv

In [86]:
U = np.array([[1, 0], [0, 1]])

compilation, gate = solovay_kitaev(U, 2)
print(gate)
print(compilation)

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
1.0
[[-3.48672027e-17-3.48672027e-17j  3.48672027e-17+3.48672027e-17j]
 [ 3.48672027e-17+3.48672027e-17j -3.48672027e-17-3.48672027e-17j]]
-3.486720268972969e-17
[[ 0.64461897+0.07213751j -0.64461897-0.07213751j]
 [-0.36704075+0.17415535j  0.36704075-0.17415535j]]
0.64461897221084
[[0.38268343+0.18979803j 0.13639057+0.40480553j]
 [0.38268343-0.50682337j 0.62897629-0.08778019j]]
0.38268343236508984
[[-3.21519374e-67+2.62748286e-67j  3.21519374e-67-2.62748286e-67j]
 [-1.47004553e-67-1.58567762e-67j  1.47004553e-67+1.58567762e-67j]]
...
